In [24]:
from __future__ import print_function

import time
import numpy as np

import scipy.misc
import scipy.io

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.applications import vgg19
from keras.models import Model
from keras.layers import Input
import tensorflow as tf

# Set backend as tensorflow
from keras import backend as K
K.set_image_dim_ordering("tf")

from PIL import Image

%matplotlib inline

In [25]:
# Reset the graph
tf.reset_default_graph()

# Start interactive session
sess = tf.InteractiveSession()
K.set_session(sess)

In [26]:
import h5py

f = h5py.File("vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5", "r")
print(f.keys())

[u'block1_conv1', u'block1_conv2', u'block1_pool', u'block2_conv1', u'block2_conv2', u'block2_pool', u'block3_conv1', u'block3_conv2', u'block3_conv3', u'block3_conv4', u'block3_pool', u'block4_conv1', u'block4_conv2', u'block4_conv3', u'block4_conv4', u'block4_pool', u'block5_conv1', u'block5_conv2', u'block5_conv3', u'block5_conv4', u'block5_pool', u'input_2']


In [27]:
STYLE_LAYERS = [
#     ('block1_conv1', 0.2),
    ('block1_conv2', 5.0),
#     ('block2_conv1', 0.2),
    ('block2_conv2', 5.0),
#     ('block3_conv1', 0.2),
#     ('block3_conv2', 0.2),
    ('block3_conv3', 5.0),
#     ('block3_conv4', 0.2),
#     ('block4_conv1', 0.2),
#     ('block4_conv2', 0.2),
#     ('block4_conv3', 0.2),
    ('block4_conv4', 5.0),
#     ('block5_conv1', 0.2),
#     ('block5_conv2', 0.2),
#     ('block5_conv3', 0.2),
    ('block5_conv4', 5.0)]

In [28]:
class CONFIG:
    IMAGE_WIDTH = 400
    IMAGE_HEIGHT = 300
    COLOR_CHANNELS = 3
    CONTENT_WEIGHT = 0.025
    STYLE_WEIGHT = 5.0
    TOTAL_VARIATION_WEIGHT = 1.0
    NOISE_RATIO = 0.6
    MEANS = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))

In [29]:
def reshape_and_normalize_image(image_path):
    """
    Reshape and normalize the input image (content or style)
    """
    
    image = Image.open(image_path)
    image = image.resize((CONFIG.IMAGE_WIDTH, CONFIG.IMAGE_HEIGHT))
    
    image_array = np.asarray(image, dtype='float32')
    image = np.expand_dims(image_array, axis=0)
    
    # Substract the mean to match the expected input of VGG16
    image = image - CONFIG.MEANS
    
    return image

In [30]:
def generate_noise_image(content_image, noise_ratio = CONFIG.NOISE_RATIO):
    """
    Generates a noisy image by adding random noise to the content_image
    """
    
    # Generate a random noise_image
    noise_image = np.random.uniform(-20, 20, (1, CONFIG.IMAGE_HEIGHT, CONFIG.IMAGE_WIDTH,
                                              CONFIG.COLOR_CHANNELS)).astype('float32')
    
    # Set the input_image to be a weighted average of the content_image and a noise_image
    input_image = noise_image * noise_ratio + content_image * (1 - noise_ratio)
    
    return input_image

In [31]:
def compute_content_loss(content, generated):
    m, n_H, n_W, n_C = generated.get_shape().as_list()

    content = tf.reshape(content, [-1])
    generated = tf.reshape(generated, [-1])

    return tf.divide(tf.reduce_sum(tf.reduce_sum(tf.square( \
                    tf.subtract(content, generated)))), (4*n_H*n_W*n_C))

In [32]:
def gram_matrix(x):
    gram = tf.matmul(x, x, transpose_b=True)
    return gram

def style_layer_loss(style, generated):
    m, n_H, n_W, n_C = generated.get_shape().as_list()
    
    style = tf.reshape(tf.transpose(style), (n_C, -1))
    generated = tf.reshape(tf.transpose(generated), (n_C, -1))
    
    S = gram_matrix(style)
    G = gram_matrix(generated)

    channels = n_C
    size = n_H*n_W

    return tf.divide(tf.reduce_sum(tf.reduce_sum(tf.square(tf.subtract(S, G)))),
                     (4. * (channels ** 2) * (size ** 2)))

In [33]:
# Get content_image from file
content_image_path = "./images/louvre_small.jpg"
content_image = reshape_and_normalize_image(content_image_path)

image_to_show = plt.imread(content_image_path)
# plt.imshow(image_to_show)

In [34]:
style_image_path = "./images/monet.jpg"
style_image = reshape_and_normalize_image(style_image_path)

image_to_show = plt.imread(content_image_path)
# plt.imshow(image_to_show)

In [35]:
# Generate a random noise_image
generated_image = generate_noise_image(content_image)

In [36]:
# Combined input for model
# input_tensor = K.concatenate([content_image,
#                           style_image,
#                           generated_image], axis=0)

input_tensor = tf.Variable(np.zeros((1,
                                     CONFIG.IMAGE_HEIGHT,
                                     CONFIG.IMAGE_WIDTH,
                                     CONFIG.COLOR_CHANNELS)),
                           dtype='float32', name="input_tensor")

input_tensor = Input(tensor=input_tensor)
model = vgg19.VGG19(weights=None,
                    input_tensor=input_tensor,
                    include_top=False)
model.load_weights("vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5")

# model.summary()

In [37]:
# get the symbolic outputs of each "key" layer (we gave them unique names).
# outputs_list = [(layer_name, model.get_layer(layer_name).output)
#                      for layer_name, _ in STYLE_LAYERS]

# layer_outputs = [output_tensors for _, output_tensors in outputs_list]
# print(layer_outputs)

In [38]:
# content_model = Model(inputs=model.input, outputs=layer_outputs)
# content_model.load_weights("vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [39]:
# model_input = model.get_layer("input_1").output
# model.summary()

In [41]:
def total_loss(alpha=, beta=40):

    # Assign the input of the model to be the "content" image 
    sess.run(input_tensor.assign(content_image))

    # Get content loss from output of block 4, layer 2
    out = model.get_layer("block4_conv2").output
    content_features = sess.run(out)
    generated_features = tf.Variable(out)
    content_loss = compute_content_loss(content_features, generated_features)
    
    # loss for style image
    style_loss = 0.
    
    # Assign the input of the model to be the "style" image 
    sess.run(input_tensor.assign(style_image))
    
    for layer_name, coeff in STYLE_LAYERS:
        # Select the output tensor of the currently selected layer
        out = model.get_layer(layer_name).output
        style_features = sess.run(out)
        generated_features = tf.Variable(out)
        style_loss += coeff * style_layer_loss(style_features, generated_features)
    
    # Get total loss using alpha and beta
    total_loss = (content_loss*alpha) + (style_loss*beta)
    
    return content_loss, style_loss, total_loss

In [42]:
content_loss, style_loss, J = total_loss()

In [43]:
# define optimizer
optimizer = tf.train.AdamOptimizer(5.0)

# define train_step
train_step = optimizer.minimize(J)

In [44]:
def save_image(path, image):
    
    # Un-normalize the image so that it looks good
    image = image + CONFIG.MEANS
    
    # Clip and Save the image
    image = np.clip(image[0], 0, 255).astype('uint8')
    scipy.misc.imsave(path, image)

In [45]:
def model_nn(sess, input_image, num_iterations = 200):
    
    # Initialize global variables (you need to run the session on the initializer)
    sess.run(tf.global_variables_initializer())
    
    # Assign the input of the model to be the "input" image 
    sess.run(input_tensor.assign(input_image))
    
    for i in range(num_iterations):
        
        # Run the session on the train_step to minimize the total cost
        sess.run([train_step])
        
        # Compute the generated image by running the session on the current model['input']
        generated_image = sess.run(input_tensor)

        # Print every 20 iteration.
        if i%20 == 0:
            Jt, Jc, Js = sess.run([J, content_loss, style_loss])
            print("Iteration " + str(i) + " :")
            print("content cost = ", Jc)
            print("style cost = ", Js)
            print("total cost = ", Jt)
            
            # save current generated image in the "/output" directory
            save_image("output/" + str(i) + ".png", generated_image)

    # save last generated image
    save_image('output/generated_image.jpg', generated_image)
    
    return generated_image

In [46]:
out = model_nn(sess, generated_image)

# print(generated_image)

Iteration 0 :
content cost =  109894.0
style cost =  3.0094e+10
total cost =  1.20376e+12


/Users/harshbhardwaj/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


Iteration 20 :
content cost =  96912.4
style cost =  3.0094e+10
total cost =  1.20376e+12
Iteration 40 :
content cost =  85800.5
style cost =  3.0094e+10
total cost =  1.20376e+12
Iteration 60 :
content cost =  76313.6
style cost =  3.0094e+10
total cost =  1.20376e+12


KeyboardInterrupt: 